# Panda IK
Do Panda IK with shoulder partitioning, using a roadmap IK module for the 3-DOF arm.

In [16]:
from pathlib import Path
import numpy as np
from typing import Optional

import gtdynamics as gtd
from gtsam import Pose3, Values, Rot3, Point3

import plotly.express as px

import roboplot  # type: ignore
%matplotlib widget

from prototype.chain import Chain

In [17]:
model_file = Path(gtd.URDF_PATH) / "panda" / "panda.urdf"
base_name = "link0"
# Crucial to fix base link or FK gives wrong result
robot = gtd.CreateRobotFromFile(str(model_file)).fixLink(base_name)

In [18]:
# Construct shoulder:
shoulder = Chain.from_robot(robot, base_name, (0, 3))


In [19]:
# Sample a few configurations:
N = 200
theta =  2 * np.pi * (np.random.random_sample((N, 3)) - 0.5)

In [20]:
# Let's plot end-effector position which is child link of last shoulder joint:
poses = [shoulder.poe(q) for q in theta]
ts = np.array([pose.translation() for pose in poses])
px.scatter_3d(x=ts[:,0], y=ts[:,1], z=ts[:,2])

In [21]:
# Solve equation for finding the shoulder "center":
A = shoulder.axes
JR = A[:3,:]
Jt = A[3:,:]
Skew = -Jt @ np.linalg.pinv(JR)
print(np.round(Skew,3))

[[ 0.    -0.249  0.039]
 [ 0.    -0.    -0.028]
 [ 0.     0.028 -0.   ]]


In [22]:
sTc = Pose3(Rot3(), Point3(-Skew[1,2], Skew[0,2], -Skew[0,1]))
np.round(sTc.AdjointMap() @ A, 5)

array([[ 0.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1., -0.,  1.],
       [-0.,  0.,  0.],
       [ 0., -0.,  0.],
       [ 0., -0.,  0.]])

In [28]:
# Let's now go to the shoulder position:
cTs = sTc.inverse()
poses = [shoulder.poe(q, cTs) for q in theta]
ts = np.array([pose.translation() for pose in poses])

In [29]:
np.round(ts[:10,:],3)

array([[ 0.   ,  0.   ,  0.333],
       [ 0.   ,  0.   ,  0.333],
       [-0.   , -0.   ,  0.333],
       [ 0.   , -0.   ,  0.333],
       [-0.   , -0.   ,  0.333],
       [ 0.   ,  0.   ,  0.333],
       [-0.   , -0.   ,  0.333],
       [-0.   , -0.   ,  0.333],
       [ 0.   ,  0.   ,  0.333],
       [-0.   ,  0.   ,  0.333]])